In [1]:
import csv
from wikimapper import WikiMapper
import rdflib
from collections import Counter

mapper = WikiMapper("index_simplewiki-20230501.db")

def best_types(qs):
    values = ['wd:' + q for q in qs]
    qres = rdflib.Graph().query(
        """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        SELECT ?type ?count
        WHERE {
          SERVICE <https://query.wikidata.org/sparql> {
            SELECT ?type (COUNT(?type) as ?count) {
              VALUES ?s { %s }
              ?s wdt:P31 ?type .
            }
            GROUP BY (?type)
          }
        } 
        """ % ' '.join(values)
    )
    return Counter({t:c.toPython() for t,c in qres})

rows = csv.reader(open('tables/table_1.csv').readlines()[1:])
cols = zip(*rows)
col_types = {}
for ci, col in enumerate(cols):
    wiki_ids = set()
    for val in set(col):
        i = mapper.title_to_id(val.replace(' ','_'))
        if i:
            wiki_ids.add(i)
    if wiki_ids:
        print(ci, wiki_ids)
        col_types[ci] = best_types(wiki_ids)

col_types

0 {'Q103916'}
2 {'Q162492', 'Q23844', 'Q166272', 'Q129817', 'Q178552', 'Q296500', 'Q171905', 'Q38111', 'Q23365', 'Q174843', 'Q134895', 'Q170587', 'Q180560', 'Q310318', 'Q40096', 'Q44221', 'Q313043', 'Q211322', 'Q171363', 'Q42869', 'Q182763', 'Q193815', 'Q173158', 'Q272019', 'Q104791', 'Q37175', 'Q123351'}



KeyboardInterrupt



In [ ]:

"""
SELECT ?p ?q ?cs ?co ?cq WHERE {
  VALUES ?cs { wd:Q5 wd:Q618779 wd:Q386724 }
  VALUES ?co { wd:Q5 wd:Q618779 wd:Q386724 }
  VALUES ?cq { wd:Q5 wd:Q618779 wd:Q386724 }
  FILTER ((?cs != ?co) && (?cs != ?cq) && (?cq != ?co))
  ?p p:P2302 [ # property constraint
    ps:P2302 wd:Q21503250; # subject type constraint
    pq:P2309 wd:Q21503252; # relation: instance of
    pq:P2308 ?cs # subject class
  ], [
    ps:P2302 wd:Q21510865; # value type constraint
    pq:P2309 wd:Q21503252; # relation: instance of
    pq:P2308 ?co # value class
  ], [ 
    ps:P2302 wd:Q21510851; # allowed qualifiers
    pq:P2306 ?q # property
  ].
  ?q p:P2302 [ # property constraint
    ps:P2302 wd:Q21510865; # value type constraint
    pq:P2309 wd:Q21503252; # relation: instance of
    pq:P2308 ?cq # qualifier class
  ].
}
"""